# Lesson 3: Chatbot Example

In this lesson, you will familiarize yourself with the chatbot example you will work on during this course. The example includes the tool definitions and execution, as well as the chatbot code. Make sure to interact with the chatbot at the end of this notebook.

## Import Libraries

In [1]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic
from IPython.display import display, Markdown

## Tool Functions

In [2]:
PAPER_DIR = "papers"

The first tool searches for relevant arXiv papers based on a topic and stores the papers' info in a JSON file (title, authors, summary, paper url and the publication date). The JSON files are organized by topics in the `papers` directory. The tool does not download the papers.  

In [3]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.

    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)

    Returns:
        List of paper IDs found in the search
    """

    # Use arxiv to find the papers
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query=topic, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)

    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "summary": paper.summary,
            "pdf_url": paper.pdf_url,
            "published": str(paper.published.date()),
        }
        papers_info[paper.get_short_id()] = paper_info

    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"Results are saved in: {file_path}")

    return paper_ids

In [4]:
search_papers("computers")

Results are saved in: papers/computers/papers_info.json


['1310.7911v2',
 'math/9711204v1',
 '2208.00733v1',
 '2504.07020v1',
 '2403.03925v1']

The second tool looks for information about a specific paper across all topic directories inside the `papers` directory.

In [5]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.

    Args:
        paper_id: The ID of the paper to look for

    Returns:
        JSON string with paper information if found, error message if not found
    """

    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue

    return f"There's no saved information related to paper {paper_id}."

In [6]:
extract_info("1310.7911v2")

'{\n  "title": "Compact manifolds with computable boundaries",\n  "authors": [\n    "Zvonko Iljazovic"\n  ],\n  "summary": "We investigate conditions under which a co-computably enumerable closed set\\nin a computable metric space is computable and prove that in each locally\\ncomputable computable metric space each co-computably enumerable compact\\nmanifold with computable boundary is computable. In fact, we examine the notion\\nof a semi-computable compact set and we prove a more general result: in any\\ncomputable metric space each semi-computable compact manifold with computable\\nboundary is computable. In particular, each semi-computable compact\\n(boundaryless) manifold is computable.",\n  "pdf_url": "http://arxiv.org/pdf/1310.7911v2",\n  "published": "2013-10-29"\n}'

## Tool Schema

Here are the schema of each tool which you will provide to the LLM.

In [7]:
tools = [
    {
        "name": "search_papers",
        "description": "Search for papers on arXiv based on a topic and store their information.",
        "input_schema": {
            "type": "object",
            "properties": {
                "topic": {"type": "string", "description": "The topic to search for"},
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve",
                    "default": 5,
                },
            },
            "required": ["topic"],
        },
    },
    {
        "name": "extract_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to look for",
                }
            },
            "required": ["paper_id"],
        },
    },
]

## Tool Mapping

This code handles tool mapping and execution.

In [8]:
mapping_tool_function = {"search_papers": search_papers, "extract_info": extract_info}


def execute_tool(tool_name, tool_args):

    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."

    elif isinstance(result, list):
        result = ", ".join(result)

    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)

    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot Code

The chatbot handles the user's queries one by one, but it does not persist memory across the queries.

In [9]:
load_dotenv()
client = anthropic.Anthropic()

### Query Processing

In [10]:
def process_query(query):

    messages = [{"role": "user", "content": query}]

    response = client.messages.create(
        max_tokens=2024,
        model="claude-3-7-sonnet-20250219",
        tools=tools,
        messages=messages,
    )

    process_query = True
    while process_query:
        assistant_content = []

        for content in response.content:
            if content.type == "text":

                display(Markdown(content.text))
                assistant_content.append(content)

                if len(response.content) == 1:
                    process_query = False

            elif content.type == "tool_use":

                assistant_content.append(content)
                messages.append({"role": "assistant", "content": assistant_content})

                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")

                result = execute_tool(tool_name, tool_args)
                messages.append(
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "tool_result",
                                "tool_use_id": tool_id,
                                "content": result,
                            }
                        ],
                    }
                )
                response = client.messages.create(
                    max_tokens=2024,
                    model="claude-3-7-sonnet-20250219",
                    tools=tools,
                    messages=messages,
                )

                if len(response.content) == 1 and response.content[0].type == "text":
                    display(Markdown(response.content[0].text))
                    process_query = False

### Chat Loop

In [11]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == "quit":
                break

            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Feel free to interact with the chatbot. Here's an example query: 

- Search for 2 papers on "LLM interpretability"

To access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L3`.

In [ ]:
chat_loop()

Type your queries or 'quit' to exit.



Query:  hello there


Hello! I'm here to help you with searching for and extracting information about research papers on arXiv. 

Is there a specific topic you're interested in learning about? Or perhaps you have a particular paper ID you'd like more information on? Let me know what you're looking for, and I can assist you with finding relevant research papers or details about a specific paper.


Query:  I'd like to know about the most impactful papers regarding LLMs


I'd be happy to help you find information about the most impactful papers regarding Large Language Models (LLMs). Let me search for relevant papers on this topic.

...

Let me try another search for more foundational LLM papers:

Calling tool search_papers with args {'topic': 'GPT-3 language models few-shot learning', 'max_results': 2}
Results are saved in: papers/gpt-3_language_models_few-shot_learning/papers_info.json
Calling tool extract_info with args {'paper_id': '2205.10981v1'}
Calling tool extract_info with args {'paper_id': '2212.00857v1'}


Let me try one more search to find the original Transformer paper:

Calling tool search_papers with args {'topic': 'Attention is All You Need', 'max_results': 1}
Results are saved in: papers/attention_is_all_you_need/papers_info.json
Calling tool extract_info with args {'paper_id': '2407.15516v1'}


Based on my searches, let me provide you with information about the most impactful papers regarding Large Language Models (LLMs):

# Most Impactful Papers on Large Language Models (LLMs)

While my search didn't surface all the original foundational papers, I can share what I found and supplement with knowledge about the most influential LLM papers:

## Foundational Papers

1. **"Attention Is All You Need" (2017)** - While not directly retrieved in my search, this paper by Vaswani et al. introduced the Transformer architecture, which is the foundation for all modern LLMs. It revolutionized NLP by replacing recurrent neural networks with attention mechanisms.

2. **"Language Models are Few-Shot Learners" (2020)** - The GPT-3 paper by Brown et al. This groundbreaking work demonstrated that scaling language models to 175 billion parameters enables few-shot learning, where models can perform tasks with minimal examples.

3. **"Training language models to follow instructions with human feedback" (2022)** - The RLHF (Reinforcement Learning from Human Feedback) paper by OpenAI that introduced the methodology behind ChatGPT, making LLMs more aligned with human intent.

## Papers Found in My Search

1. **"A survey on GPT-3" (2022)** - By Zong and Krishnamachari, this paper provides an overview of GPT-3's capabilities, applications across domains, and challenges it faces. It documents the impact of one of the most influential LLMs.

2. **"Lost in Translation: Large Language Models in Non-English Content Analysis" (2023)** - This paper highlights the challenges of LLMs in languages other than English and discusses multilingual language models.

3. **"Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models" (2024)** - A recent paper showing how LLM inference can be optimized by dropping certain attention layers with minimal performance impact.

4. **"Goldfish: Monolingual Language Models for 350 Languages" (2024)** - Addresses the challenge of creating language models for low-resource languages, an important direction for making LLM technology more inclusive.

## Key Impact Areas

The most impactful LLM papers have contributed to:

1. **Architecture Innovation**: The Transformer architecture revolutionized how we process sequential data.

2. **Scaling Laws**: Papers demonstrating how model performance improves with increased parameters, data, and compute.

3. **Emergent Abilities**: Research showing how capabilities like reasoning, in-context learning, and instruction following emerge at scale.

4. **Alignment and Safety**: Methods to align LLMs with human values and reduce harmful outputs.

5. **Multimodality**: Extending LLMs to work with images, audio, and other modalities.

Would you like me to search for more specific foundational papers or focus on a particular aspect of LLM research?

In [24]:
!ls papers/*

papers/attention_is_all_you_need:
papers_info.json

papers/computers:
papers_info.json

papers/gpt-3_language_models_few-shot_learning:
papers_info.json

papers/large_language_models:
papers_info.json

papers/transformer_attention_is_all_you_need:
papers_info.json


<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> To Access the <code>requirements.txt</code> file or the <code>papers</code> folder: </b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em> and finally 3) click on <em>"L3"</em>.
</div>

In the next lessons, you will take out the tool definitions to wrap them in an MCP server. Then you will create an MCP client inside the chatbot to make the chatbot MCP compatible.  

## Resources

[Guide on how to implement tool use](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#how-to-implement-tool-use)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">


<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>